In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog,dog_box, hidden_state, bfsp, sp) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.gru_cell = hidden_state.float().to('cuda:0')
        self.visited = 0
        self.bfsp = bfsp
        self.gru_cell_out = None
        
    def lstm_i(self, hidden_state):
        self.gru_cell = hidden_state
        self.visited = self.visited + 1

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        hidden_state = lstm_o
        self.gru_cell_out = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i(hidden_state) #((lh.detach(), lc.detach())) #DETACH

    def detach_state(self):
        self.gru_cell = self.gru_cell.detach()
            


In [6]:
class Dog:
    def __init__(self, dogid,dog_name, hidden_size, layers) -> None:
        self.dogid = dogid
        self.dog_name = dog_name
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid, box, bfsp=None, sp=None):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, box, torch.randn(self.hidden_size), bfsp, sp) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [7]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes=None):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.track_name = None
        self.raw_margins = None
        self.raw_places = None
        self.raw_prices = None
        
        if classes!=None:
            self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def add_track_name(self, track_name):
        self.track_name = track_name


    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self, pred=False):
        if pred:
            print('pred')
        else:
            l_input = [x.gru_cell for x in self.dogs]
        return l_input

    def lstm_detach(self):
        [x.detach_state for x in self.dogs]

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_gru_output(self, hidden_states):
        for i,dog in enumerate(self.dogs):
            hs = hidden_states[i]
            #hs = hs.detach()
            dog.lstm_o(hs) #.clone())
            
    def add_weights(self, weights):
        self.weights = weights

    def add_win_weight(self):
        pass

In [43]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes=None):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid, dog_name):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid,dog_name, self.hidden_size, self.layers)

    def create_test_split(self):
        self.train_race_ids = []
        self.test_race_ids = []
        for i,r in enumerate(self.raceIDs):
            if i%10>(8):
                self.test_race_ids.append(r)
            else:
                self.train_race_ids.append(r)


    def get_race_input(self, idx, Train=True) -> Race:
        if len(idx)==1:
            race = self.racesDict[self.raceIDs[idx]]
            print(f"returing race {race}")
            return race
        else:
            raceidx = operator.itemgetter(*idx)
            race_batch_id = raceidx(self.raceIDs)
            races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        
        return races #self.racesDict[raceidx]

    def get_train_input(self, idx, Train=True) -> Race:
        raceidx = operator.itemgetter(*idx)
        race_batch_id = raceidx(self.train_race_ids)
        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        return races #self.racesDict[raceidx]

    def get_test_input(self, idx, Train=True) -> Race:
        raceidx = operator.itemgetter(*idx)
        race_batch_id = raceidx(self.test_race_ids)
        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        return races #self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes
        

    def get_race_weights(self, races):
        print("weights")

    def reset_all_lstm_states(self):
        for race in self.racesDict.values:
            for dog in race.dogs:
                dog.lstmCellc = torch.rand(self.hidden_size)
                dog.lstmCellh = torch.rand(self.hidden_size)
                dog.gru_cell = torch.rand(self.hidden_size)

    def detach_all_hidden_states(self):
        for race in self.racesDict.values():
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def detach_given_hidden_states(self, races):
        for race in races:
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def create_hidden_states_dict(self):
        self.hidden_states_dict = {}
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                key = race_id+'_'+dog_id
                val = dog.gru_cell_out
                self.hidden_states_dict[key] = val

    def fill_hidden_states_from_dict(self, hidden_dict):
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                dog_prev_race_id = dog.prevrace_id
                key = str(dog_prev_race_id)+'_'+dog_id
                try:
                    val = hidden_dict[key]
                    if val != None:
                        dog.gru_cell = val
                    else:
                        dog.gru_cell = torch.rand(self.hidden_size)
                except KeyError as e:
                    print(f'Key error {e}')
                    val = torch.rand(self.hidden_size)
                    dog.gru_cell = val
                    print(f"race in = {dog.gru_cell}")
                print(key,val)

    def to_cuda(self):
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.to(device)

    def race_prices_to_prob(self):
        for r in self.racesDict.values():
            r.implied_prob = [1/(x+0.0001) for x in r.prices]



In [9]:
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.3):
        super(GRUNet, self).__init__()
        self.gru1 = nn.GRUCell(input_size, hidden_size)
        self.gru2 = nn.GRUCell(input_size, hidden_size)
        self.gru3 = nn.GRUCell(input_size, hidden_size)
        self.gru4 = nn.GRUCell(input_size, hidden_size)
        self.gru5 = nn.GRUCell(input_size, hidden_size)
        self.gru6 = nn.GRUCell(input_size, hidden_size)
        self.gru7 = nn.GRUCell(input_size, hidden_size)
        self.gru8 = nn.GRUCell(input_size, hidden_size)
        self.rl1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.drop2 = nn.Dropout(dropout)
        self.rl2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

        self.lsftmax = nn.LogSoftmax(dim=1)

    # x represents our data
    def forward(self, race: Race)  :
        x = torch.stack([r.full_input.float() for r in race])#.detach()

        # creates list of LSTM data
        hidden_state_in = [list(i) for i in zip(*[r.lstm_input() for r in race])]

        # creates list of tensors for lstm Cells
        hCell = [torch.stack([x for x in y]) for y in hidden_state_in]

        h1 = self.gru1(x, hCell[0])
        h2 = self.gru2(x, hCell[1])
        h3 = self.gru3(x, hCell[2])
        h4 = self.gru4(x, hCell[3])
        h5 = self.gru5(x, hCell[4])
        h6 = self.gru6(x, hCell[5])
        h7 = self.gru7(x, hCell[6])
        h8 = self.gru8(x, hCell[7])

        lstm_list = [
            h1,
            h2,
            h3,
            h4,
            h5,
            h6,
            h7,
            h8,
        ]

        hCello = [i for i in zip(*[x for x in lstm_list])]
        # cCello = [i for i in zip(*[x[1] for x in lstm_list])]

        for i, r in enumerate(race):
            r.pass_gru_output(hCello[i])
        xhh = torch.cat((h1, h2, h3, h4, h5, h6, h7, h8), dim=1)
        xr1 = self.rl1(xhh)
        xd1 = self.drop1(xr1)
        xh = self.fc2(xd1)
        xd2 = self.drop2(xh)
        xr2 = self.rl2(xd2)
        xf = self.fc3(xr2)

        # output = F.softmax(xf, dim=1)
        output = self.lsftmax(xf)
        # output = xf
        return output


In [10]:
def build_pred_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])

    for i,j in tqdm(dog_stats_group):
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),-1, x['prev_race'], x['box']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=0)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED

    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])

        empty_dog_list = [null_dog_i]*8
        boxes_list = [x for x in j['box']]      
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        for n,x in enumerate(boxes_list):
            empty_dog_list[x-1] = dog_list[n]
        
        raceDB.add_race(i,trackOHE,dist)
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].add_track_name(j.track_name.iloc[0])
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    return raceDB



In [37]:
def build_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])
    dog_stats_df['runtime'] = pd.to_numeric(dog_stats_df['runtime'])

    dog_stats_df = dog_stats_df[dog_stats_df['track_name'].str.contains("NZ")].reset_index()

    #Generate weights for classes per track:

    grouped = dog_stats_df.groupby('track_name')
    track_weights = {}

    for i,j in grouped:
        weights = (1-(j[j['place']==1]['box'].value_counts(sort=False)/len(j[j['place']==1]))).tolist()
        if len(weights) !=8:
            weights.append(0)
        track_weights[i] = torch.tensor(weights).to(device)

    #track_weights['Cranbourne'] = track_weights['Cranbourne']*5
    

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    num_features_per_dog = len(dog_stats_df['stats'][0])
    print(f"{num_features_per_dog=}")

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"]) 

    for i,j in tqdm(dog_stats_group):
        j["next_race"] = j["raceid"].shift(-1).fillna(-1)
        j["prev_race"] = j["raceid"].shift(1).fillna(-1)
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race'], x['box'], x['bfSP'], x['startprice']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=1)
    #lsoftmax = F.log_softmax(dim=1)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(num_features_per_dog), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    #TO FIX LATER PROPER BOX PLACEMENT #FIXED
    dog_stats_df = dog_stats_df.sort_values('date')
    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])
        #margins
        empty_dog_list = [null_dog_i]*8
        empty_margin_list = [-20]*8
        empty_log_margin_list = [3]*8
        empty_place_list = [8]*8
        empty_finish_list = [40]*8
        empty_price_list = [0]*8
        untouched_margin = [20]*8

        places_list = [x for x in j["place"]]
        boxes_list = [x for x in j['box']]
        margin_list = [x for x in j["margin"]]
        time_list = [x for x in j["runtime"]]
        price_list = [x for x in j['bfSP'].astype(float)]

        # empty_log_margin_list = np.log(max(margin_list)+1)
        
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        #adjustedMargin = [margin_list[x-1] for x in boxes_list]
        for n,x in enumerate(boxes_list):
            empty_margin_list[x-1] = -(margin_list[n]+5)
            empty_log_margin_list[x-1] = margin_list[n]+1
            empty_dog_list[x-1] = dog_list[n]
            empty_place_list[x-1] = places_list[n]
            empty_finish_list[x-1] = time_list[n]
            empty_price_list[x-1] = price_list[n]
            untouched_margin[x-1] = margin_list[n]
        # adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        # adjustedMargin = torch.log((torch.tensor(empty_log_margin_list).log()).to('cuda:0'))
        adjustedMargin = F.log_softmax(torch.tensor(empty_margin_list), dim=-1).to('cuda:0')
        # adjustedMargin = softmin(torch.tensor(empty_finish_list)).to('cuda:0')
        # adjustedMargin = softmin(torch.tensor(empty_place_list)).to('cuda:0')
        # adjustedMargin = torch.tensor(empty_place_list).to('cuda:0')
        #adjusted_dog_list = [dog_list[x-1] for x in boxes_list]
        
        raceDB.add_race(i,trackOHE,dist, adjustedMargin)
        try:
            dog_win_box = j[j['place']==1]['box'][0].item()
        except Exception as e:
            dog_win_box = 1
        raceDB.racesDict[i].win_weight = track_weights[j.track_name.iloc[0]][dog_win_box-1]
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]
        raceDB.racesDict[i].weights = track_weights[j.track_name.iloc[0]]
        raceDB.racesDict[i].raw_margins = empty_margin_list
        raceDB.racesDict[i].raw_places = empty_place_list
        raceDB.racesDict[i].untouched_margin = untouched_margin
        raceDB.racesDict[i].prices = empty_price_list

    raceDB.race_prices_to_prob()

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    raceDB.create_test_split()
    return raceDB


In [12]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    model_name = wandb.run.name
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [13]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [76]:
#Testing
def validate_model(model:GRUNet,raceDB,criterion, batch_size, example_ct, epoch_loss, batch_ct):
    torch.autograd.set_detect_anomaly(True)
    sft_max = nn.Softmax(dim=-1)
    batch_size=100
    len_test = len(raceDB.test_race_ids)-batch_size
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    loss_l = []
    loss_t = []
    margins_l = []
    preds_l = []
    pred_sftmax = []
    raw_margins = []
    raw_places = []

    prices_list = []
    raw_prices = []

    price_dict = {}
    price_dict['prices'] = []
    price_dict['imp_prob'] = []
    price_dict['pred_prob'] = []
    price_dict['pred_price'] = []
    price_dict['margin'] = []
    print("starting acc")
    with torch.no_grad():
        for i in trange(0,len_test,batch_size, leave=False):
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_test_input(races_idx)
            #tracks.extend([r.track for r in race])
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)

            _, actual = torch.max(y.data, 1)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()

            softmax_preds = torch.exp(output).tolist()
            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)

            
            loss = criterion(output, y).detach()
            loss_tensor = validation_CLE(output,y)
            loss_t.append(loss_tensor.tolist())

            loss_l.append(loss.tolist())
            preds_l.append(output.tolist())
            pred_sftmax.append(softmax_preds)
            margins_l.append(y.tolist())
            raw_margins.append([x.raw_margins for x in race])
            raw_places.append([x.raw_places for x in race])
            loss_val += loss

            price_dict['prices'].extend([x.prices for x in race])
            price_dict['imp_prob'].extend([x.implied_prob for x in race])
            price_dict['pred_prob'].extend(softmax_preds)
            #print([(1/(x+(-7**10))).tolist() for x in torch.exp(output)])
            price_dict['pred_price'].extend([(1/(x)).tolist() for x in torch.exp(output)])
            price_dict['margin'].extend([x.raw_margins for x in race])

            

        loss_list = []

        #print("start loss calc")
        for i,l in enumerate(loss_l):
            for j in range(0,7):
                loss_list.append([preds_l[i][j],margins_l[i][j],loss_t[i][j],l[j],pred_sftmax[i][j], raw_margins[i][j], raw_places[i][j]])

    loss_df = pd.DataFrame(loss_list, columns=['Preds','Margins','Indiviual Losses','Losses','softmaxPreds','Raw margins', 'Raw places'])
    loss_table = wandb.Table(dataframe=loss_df)
    # try:
    #     wandb.log({"loss_table": loss_table})
    # except Exception as e:
    #     print(e)

    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })
    
    logdf['correct'] = logdf.apply(lambda x: 1 if x['actuals']==x['preds'] else 0, axis=1)
    # logdf['return'] = logdf.apply(lambda x: x['bfsps']-1 if x['correct'] else 0 if x['bfsps']<1 else -1)
    logdf['profit'] = logdf.apply(lambda x: 0 if x['bfsps']<1 else x['bfsps']-1  if x['correct'] else -1, axis=1)
    logdf.to_csv('logDFtest.csv')

    table = wandb.Table(dataframe=logdf)
    # try:
    #     wandb.log({"table_key": table})
    # except Exception as e:
    #     print(e)
    logdf['count'] = 1
    track_df = logdf.groupby('track', as_index=False).sum().reset_index()
    track_table = wandb.Table(dataframe=track_df)
    #wandb.log({"track_df": track_table})
    
    prices_df = pd.DataFrame(price_dict)
    price_table = wandb.Table(dataframe=prices_df)
    #wandb.log({"price_table":price_table})
    
    
    try:
        wandb.log({"table_key": table,"loss_table": loss_table,"track_df": track_table,"price_table":price_table })
    except Exception as e:
        print(e)

    print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": torch.mean(loss_val)/len_test, "correct": correct, 'profit': logdf[logdf['actuals']==logdf['preds']]['profit'].sum()})

    return correct/total

In [54]:
x = torch.rand(10,5)
x.tolist()

[[0.440761923789978,
  0.6743767261505127,
  0.41654324531555176,
  0.3431577682495117,
  0.5345848798751831],
 [0.3367096781730652,
  0.5411312580108643,
  0.304157555103302,
  0.8637804388999939,
  0.9805387258529663],
 [0.4273719787597656,
  0.8843240141868591,
  0.9309478402137756,
  0.33693522214889526,
  0.9122727513313293],
 [0.4117048978805542,
  0.6796284317970276,
  0.8867889046669006,
  0.39462441205978394,
  0.35012364387512207],
 [0.9304608702659607,
  0.9664019346237183,
  0.5469847917556763,
  0.2590172290802002,
  0.7750930786132812],
 [0.7583472728729248,
  0.5847869515419006,
  0.6911636590957642,
  0.42768383026123047,
  0.7482738494873047],
 [0.8184000849723816,
  0.48018938302993774,
  0.24905908107757568,
  0.3955298662185669,
  0.23600345849990845],
 [0.13232040405273438,
  0.4835861325263977,
  0.9519836902618408,
  0.6179119348526001,
  0.4155760407447815],
 [0.7945501208305359,
  0.3098054528236389,
  0.3304484486579895,
  0.8877806067466736,
  0.9504178762435

In [58]:
[(1/x).tolist() for x in x]

[[2.268798589706421,
  1.4828506708145142,
  2.4007110595703125,
  2.914111614227295,
  1.8706103563308716],
 [2.9699175357818604,
  1.8479804992675781,
  3.2877697944641113,
  1.1577016115188599,
  1.0198475122451782],
 [2.3398818969726562,
  1.1308072805404663,
  1.0741740465164185,
  2.9679296016693115,
  1.0961633920669556],
 [2.428924322128296,
  1.471392273902893,
  1.1276640892028809,
  2.53405499458313,
  2.8561339378356934],
 [1.07473623752594,
  1.0347661972045898,
  1.8282043933868408,
  3.8607470989227295,
  1.2901675701141357],
 [1.3186570405960083,
  1.7100244760513306,
  1.4468352794647217,
  2.3381757736206055,
  1.336409091949463],
 [1.2218962907791138,
  2.0825116634368896,
  4.015111446380615,
  2.5282540321350098,
  4.2372260093688965],
 [7.557413578033447,
  2.0678839683532715,
  1.0504381656646729,
  1.6183536052703857,
  2.4062983989715576],
 [1.2585738897323608,
  3.227832078933716,
  3.0261905193328857,
  1.1264044046401978,
  1.0521687269210815],
 [2.268357992

In [15]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)

In [16]:
def train(model, raceDB:Races, criterion, optimizer,scheduler, config=None):
    torch.autograd.set_detect_anomaly(True)

    last = 0
    batch_size = 25
    len_train = len(raceDB.train_race_ids)-batch_size
    example_ct = 0  # number of examples seen
    batch_ct = 0
    m = nn.LogSoftmax(dim=1)
    epoch_loss=0

    #losses = torch.tesnor()
    for epoch in trange(2000):
        model.train()
        batch_ct = 0
        setup_loss = 1
        
        for i in trange(0,len_train,batch_size, leave=False):
            last = i
            #print(f"{i=}\n{batch_ct=}, {setup_loss=}, {batch_ct+1%10==0=}")
            if ((batch_ct+1)%10==0):
                if setup_loss:
                    print("hit here")
                    continue
                else:
                    epoch_loss = torch.mean(epoch_loss)
                    optimizer.zero_grad()
                    epoch_loss.backward()
                    optimizer.step()
                    model.zero_grad()
                    raceDB.detach_all_hidden_states()
                    setup_loss = 1
                    wandb.log({"batch_loss": epoch_loss}, step = example_ct)

            batch_ct += 1

            races_idx = range(last,last+batch_size)
            race = raceDB.get_train_input(races_idx)
            X = race
            y = torch.stack([x.classes for x in race])
            w = torch.stack([x.weights for x in race])
            _, actual = torch.max(y.data, 1)
            output = model(X)

            
            example_ct +=  batch_size

            loss = criterion(output, y)*w
            if setup_loss:
                epoch_loss = loss
                setup_loss=0

            if ((batch_ct + 1) % 25) == 0:                    
                    train_log(torch.mean(loss), example_ct, epoch)

            #epoch_loss = torch.stack([epoch_loss, loss])
            epoch_loss = epoch_loss + loss
            

        print("finished epoch")
        setup_loss = 1

        wandb.log({"epoch_loss": torch.mean(epoch_loss)}, step = example_ct)
        acc = validate_model(model,raceDB,criterion, 8, example_ct, epoch_loss, batch_ct)
        epoch_loss = 0  
        print(acc)
        scheduler.step(acc)
    #print(losses)
    return model



In [44]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df_FASTTRACK new margins.npy', 'rb')
hidden_size = 64
raceDB = build_dataset(dog_stats_file, hidden_size)

num_features_per_dog=16


  0%|          | 0/3332 [00:00<?, ?it/s]

  0%|          | 0/14709 [00:00<?, ?it/s]

number of races = 14709, number of unique dogs = 3332


In [47]:
raceDB.get_race_input([1,2,500])[-1].prices

[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]

In [18]:

#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
hidden_size = 64
predDB = build_pred_dataset(dog_stats_file, hidden_size)

  0%|          | 0/363 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

number of races = 48, number of unique dogs = 363


In [19]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)
# len(raceDB.hidden_states_dict)
# predDB.to_cuda()

In [20]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [21]:
print(os.getcwd())
#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\dog_stats_df_FASTTRACK new margins.npy", 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)
dog_stats_df = dog_stats_df.sort_values('date')
dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])
dog_stats_df
weights = (1-(dog_stats_df[dog_stats_df['place']==1]['box'].value_counts(sort=False)/len(dog_stats_df[dog_stats_df['place']==1]))).tolist()
weights = torch.tensor(weights).to(device)
weights

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


tensor([0.8130, 0.8508, 0.8918, 0.8742, 0.9023, 0.9040, 0.8893, 0.8746],
       device='cuda:0')

In [22]:
dog_stats_df[dog_stats_df['track_name'].str.contains("NZ")]

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime,box
1078,2105680102,AGBEZE,500543152,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland (NZ),318.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,NaN,9.26,NaN,19.42,6
1077,183877600,RAINY RIVER,500543152,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland (NZ),318.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",1.0,NaN,0.00,NaN,18.84,5
1076,393648741,BIG TIME ROSE,500543152,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland (NZ),318.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",2.0,NaN,2.73,NaN,19.01,2
1075,262075111,BIGTIME GAL,500543152,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland (NZ),318.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",3.0,NaN,2.99,NaN,19.03,4
1074,327378038,BLACK BRIDGE,500543152,Class 1,2019-12-01,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Auckland (NZ),318.0,"[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,NaN,3.69,NaN,19.07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012523,515901776,GOLDSTAR JUSTICE,846606727,Class 3,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[6, 0.06028669106317412, 0.0007605481469190106...",6.0,47.2,6.80,NaN,17.69,6
1012521,638803656,SMASH WAVE,846606727,Class 3,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[2, 0.05984987893462469, 0.0011012205807250535...",3.0,2.5,3.00,NaN,17.45,2
1012520,529115630,MACCY'S GIRL,846606727,Class 3,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[8, 0.05980252531352187, 0.001157418694075203,...",2.0,5.8,2.70,NaN,17.43,8
1012519,638803606,HOMEBUSH ROCKET,846606727,Class 3,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Christchurch (NZ),295.0,"[1, 0.059773388381862955, 0.000668270228663087...",1.0,4.4,0.00,NaN,17.26,1


In [23]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="GRU - FastTrack", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      model = GRUNet(203,config['hidden_size'])
      criterion = nn.KLDivLoss(reduction='none', log_target=True)
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',threshold=0.001, patience=50, verbose=True, factor=0.5)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        train(model, dataset, criterion, optimizer, scheduler, config)
      except KeyboardInterrupt:
        print("finished Early")
      dataset.create_hidden_states_dict()
      # if sweep:
    #   raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset)

In [24]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}

In [25]:
raceDB.detach_all_hidden_states()

In [26]:
raceDB.create_test_split()

In [27]:
print(raceDB)

In [28]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": 70000},
    "hidden_size": {"value":16}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [500,1000,5000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [500, 1000, 5000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 16},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 0.00011},
                'len_data': {'value': 70000},
                'loss': {'values': ['L1']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 70000},
  'hidden_size': {'value': 16},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['L1']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 0.00011, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [500, 1000, 5000]}}}

In [29]:
os.getcwd()
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

'C:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\DATA'

In [77]:
(model,dataset) = model_pipeline(raceDB,config=wandb_config_static, prev_model=model)

{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(203, 64)
  (gru2): GRUCell(203, 64)
  (gru3): GRUCell(203, 64)
  (gru4): GRUCell(203, 64)
  (gru5): GRUCell(203, 64)
  (gru6): GRUCell(203, 64)
  (gru7): GRUCell(203, 64)
  (gru8): GRUCell(203, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (lsftmax): LogSoftmax(dim=1)
)


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/529 [00:00<?, ?it/s]

finished epoch
starting acc


  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/529 [00:00<?, ?it/s]

finished epoch
starting acc


  0%|          | 0/14 [00:00<?, ?it/s]

accuray: 0.21857142857142858
0.21857142857142858


  0%|          | 0/529 [00:00<?, ?it/s]

In [ ]:
stop

NameError: name 'stop' is not defined

In [70]:
test_race = raceDB.get_race_input([100,200])

AttributeError: 'list' object has no attribute 'raw_prices'

In [72]:
output = model(test_race)
output

tensor([[-1.7781, -1.9736, -1.9787, -1.9268, -2.2372, -2.4047, -2.3061, -2.1934],
        [-1.7417, -1.9999, -2.0675, -2.2382, -2.1880, -2.2963, -2.1741, -2.0417]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [75]:
[(1/x).tolist() for x in torch.exp(output)]

[[5.918612957000732,
  7.196648120880127,
  7.233370780944824,
  6.867717742919922,
  9.367378234863281,
  11.075141906738281,
  10.035475730895996,
  8.965489387512207],
 [5.70729398727417,
  7.388182640075684,
  7.9051032066345215,
  9.3767671585083,
  8.917766571044922,
  9.937180519104004,
  8.793951988220215,
  7.703883171081543]]

In [ ]:

dataset.create_hidden_states_dict()

In [ ]:
predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)